# Loop 51 Analysis: Submission Error Investigation

**Problem:** exp_050 submission failed with "Evaluation metric raised an unexpected error"

**Goal:** Understand what's wrong with the submission format and fix it.

In [1]:
import pandas as pd
import numpy as np

# Load our submission
sub = pd.read_csv('/home/submission/submission.csv')
print('Submission shape:', sub.shape)
print('Columns:', list(sub.columns))
print()
print('First 10 rows:')
print(sub.head(10))

Submission shape: (1883, 8)
Columns: ['id', 'index', 'task', 'fold', 'row', 'target_1', 'target_2', 'target_3']

First 10 rows:
   id  index  task  fold  row  target_1  target_2  target_3
0   0      0     0     0    0  0.016488  0.026118  0.897280
1   1      1     0     0    1  0.023463  0.033189  0.879097
2   2      2     0     0    2  0.039003  0.051166  0.847558
3   3      3     0     0    3  0.065632  0.074234  0.780666
4   4      4     0     0    4  0.084883  0.100093  0.714652
5   5      5     0     0    5  0.100112  0.115741  0.680016
6   6      6     0     0    6  0.100376  0.121476  0.687037
7   7      7     0     0    7  0.096191  0.120146  0.694457
8   8      8     0     0    8  0.102156  0.111917  0.673413
9   9      9     0     0    9  0.102156  0.111917  0.673413


In [2]:
# Check the data types
print('Data types:')
print(sub.dtypes)
print()

# Check for any issues
print('NaN values:')
print(sub.isnull().sum())
print()

print('Infinite values:')
for col in ['target_1', 'target_2', 'target_3']:
    print(f'  {col}: {np.isinf(sub[col]).sum()}')
print()

print('Negative values:')
for col in ['target_1', 'target_2', 'target_3']:
    print(f'  {col}: {(sub[col] < 0).sum()}')

Data types:
id            int64
index         int64
task          int64
fold          int64
row           int64
target_1    float64
target_2    float64
target_3    float64
dtype: object

NaN values:
id          0
index       0
task        0
fold        0
row         0
target_1    0
target_2    0
target_3    0
dtype: int64

Infinite values:
  target_1: 0
  target_2: 0
  target_3: 0

Negative values:
  target_1: 0
  target_2: 0
  target_3: 0


In [3]:
# Check the row counts per fold
print('Task 0 (single solvent) - rows per fold:')
task0 = sub[sub['task'] == 0]
print(task0.groupby('fold').size())
print(f'Total: {len(task0)}')
print()

print('Task 1 (full data) - rows per fold:')
task1 = sub[sub['task'] == 1]
print(task1.groupby('fold').size())
print(f'Total: {len(task1)}')

Task 0 (single solvent) - rows per fold:
fold
0     37
1     37
2     58
3     59
4     22
5     18
6     34
7     41
8     20
9     22
10    18
11    18
12    42
13    18
14    17
15    22
16     5
17    16
18    36
19    18
20    21
21    22
22    37
23    18
dtype: int64
Total: 656

Task 1 (full data) - rows per fold:
fold
0     124
1     125
2     124
3     125
4     104
5     110
6      36
7     127
8      34
9     122
10     35
11    125
12     36
dtype: int64
Total: 1227


In [4]:
# Load the actual data to compare
DATA_PATH = '/home/data'
df_single = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
df_full = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')

print('Actual data:')
print(f'  Single solvent: {len(df_single)} rows, {df_single["SOLVENT NAME"].nunique()} solvents')
print(f'  Full data: {len(df_full)} rows, {len(df_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates())} solvent pairs')
print()

# Check if our submission has the right number of rows
print('Our submission:')
print(f'  Task 0: {len(task0)} rows (expected: {len(df_single)})')
print(f'  Task 1: {len(task1)} rows (expected: {len(df_full)})')
print(f'  Total: {len(sub)} rows (expected: {len(df_single) + len(df_full)})')

Actual data:
  Single solvent: 656 rows, 24 solvents
  Full data: 1227 rows, 13 solvent pairs

Our submission:
  Task 0: 656 rows (expected: 656)
  Task 1: 1227 rows (expected: 1227)
  Total: 1883 rows (expected: 1883)


In [5]:
# Check the row indices per fold
print('Task 0 - row indices per fold:')
for fold in sorted(task0['fold'].unique()):
    fold_data = task0[task0['fold'] == fold]
    print(f'  Fold {fold}: rows {fold_data["row"].min()}-{fold_data["row"].max()} (n={len(fold_data)})')
print()

print('Task 1 - row indices per fold:')
for fold in sorted(task1['fold'].unique()):
    fold_data = task1[task1['fold'] == fold]
    print(f'  Fold {fold}: rows {fold_data["row"].min()}-{fold_data["row"].max()} (n={len(fold_data)})')

Task 0 - row indices per fold:
  Fold 0: rows 0-36 (n=37)
  Fold 1: rows 0-36 (n=37)
  Fold 2: rows 0-57 (n=58)
  Fold 3: rows 0-58 (n=59)
  Fold 4: rows 0-21 (n=22)
  Fold 5: rows 0-17 (n=18)
  Fold 6: rows 0-33 (n=34)
  Fold 7: rows 0-40 (n=41)
  Fold 8: rows 0-19 (n=20)
  Fold 9: rows 0-21 (n=22)
  Fold 10: rows 0-17 (n=18)
  Fold 11: rows 0-17 (n=18)
  Fold 12: rows 0-41 (n=42)
  Fold 13: rows 0-17 (n=18)
  Fold 14: rows 0-16 (n=17)
  Fold 15: rows 0-21 (n=22)
  Fold 16: rows 0-4 (n=5)
  Fold 17: rows 0-15 (n=16)
  Fold 18: rows 0-35 (n=36)
  Fold 19: rows 0-17 (n=18)
  Fold 20: rows 0-20 (n=21)
  Fold 21: rows 0-21 (n=22)
  Fold 22: rows 0-36 (n=37)
  Fold 23: rows 0-17 (n=18)

Task 1 - row indices per fold:
  Fold 0: rows 0-123 (n=124)
  Fold 1: rows 0-124 (n=125)
  Fold 2: rows 0-123 (n=124)
  Fold 3: rows 0-124 (n=125)
  Fold 4: rows 0-103 (n=104)
  Fold 5: rows 0-109 (n=110)
  Fold 6: rows 0-35 (n=36)
  Fold 7: rows 0-126 (n=127)
  Fold 8: rows 0-33 (n=34)
  Fold 9: rows 0-121

In [6]:
# Compare with a successful submission format
# Let's check what the template produces

# The template shows:
# - id: row index (0 to N-1)
# - index: same as id (from reset_index())
# - task: 0 for single solvent, 1 for full data
# - fold: fold index
# - row: row index within the fold
# - target_1, target_2, target_3: predictions

# Our submission has the same format
print('Checking format consistency...')
print(f'id == index: {(sub["id"] == sub["index"]).all()}')
print(f'id is sequential: {(sub["id"] == range(len(sub))).all()}')
print(f'task values: {sorted(sub["task"].unique())}')
print(f'fold values for task 0: {sorted(task0["fold"].unique())}')
print(f'fold values for task 1: {sorted(task1["fold"].unique())}')

Checking format consistency...
id == index: False
id is sequential: True
task values: [0, 1]
fold values for task 0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
fold values for task 1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [7]:
# Check if the row indices are correct
# For each fold, row should start from 0
print('Checking row indices...')
for task in [0, 1]:
    task_data = sub[sub['task'] == task]
    for fold in sorted(task_data['fold'].unique()):
        fold_data = task_data[task_data['fold'] == fold]
        if fold_data['row'].min() != 0:
            print(f'  Task {task}, Fold {fold}: row starts at {fold_data["row"].min()} (should be 0)')
        if fold_data['row'].max() != len(fold_data) - 1:
            print(f'  Task {task}, Fold {fold}: row ends at {fold_data["row"].max()} (should be {len(fold_data) - 1})')
print('Done checking row indices.')

Checking row indices...
Done checking row indices.


In [8]:
# CV-LB relationship analysis
print('='*60)
print('CV-LB RELATIONSHIP ANALYSIS')
print('='*60)

# Historical submissions
submissions = [
    ('exp_000', 0.0111, 0.0982),
    ('exp_001', 0.0123, 0.1065),
    ('exp_003', 0.0105, 0.0972),
    ('exp_005', 0.0104, 0.0969),
    ('exp_006', 0.0097, 0.0946),
    ('exp_007', 0.0093, 0.0932),
    ('exp_009', 0.0092, 0.0936),
    ('exp_012', 0.0090, 0.0913),
    ('exp_024', 0.0087, 0.0893),
    ('exp_026', 0.0085, 0.0887),
    ('exp_030', 0.0083, 0.0877),
    ('exp_035', 0.0098, 0.0970),
]

from scipy import stats

cv_scores = np.array([s[1] for s in submissions])
lb_scores = np.array([s[2] for s in submissions])

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(cv_scores, lb_scores)

print(f'Linear fit: LB = {slope:.4f} * CV + {intercept:.4f}')
print(f'R-squared = {r_value**2:.4f}')
print(f'Intercept = {intercept:.4f}')
print()

# Target analysis
target = 0.0347
print(f'Target LB: {target}')
print(f'Required CV to hit target: (target - intercept) / slope = ({target} - {intercept:.4f}) / {slope:.4f} = {(target - intercept) / slope:.6f}')
print()

# Predicted LB for exp_050
cv_050 = 0.008092
predicted_lb = slope * cv_050 + intercept
print(f'exp_050 CV: {cv_050}')
print(f'Predicted LB: {predicted_lb:.4f}')
print(f'Best LB so far: 0.0877 (exp_030)')

CV-LB RELATIONSHIP ANALYSIS


Linear fit: LB = 4.3147 * CV + 0.0525
R-squared = 0.9505
Intercept = 0.0525

Target LB: 0.0347
Required CV to hit target: (target - intercept) / slope = (0.0347 - 0.0525) / 4.3147 = -0.004130

exp_050 CV: 0.008092
Predicted LB: 0.0874
Best LB so far: 0.0877 (exp_030)


In [9]:
# Key insight: The intercept problem
print('='*60)
print('KEY INSIGHT: THE INTERCEPT PROBLEM')
print('='*60)
print()
print(f'Intercept: {intercept:.4f}')
print(f'Target: {target}')
print()
if intercept > target:
    print('CRITICAL: The intercept is HIGHER than the target!')
    print('This means even with CV=0 (perfect training), LB would be ~{:.4f}'.format(intercept))
    print('The target is BELOW the intercept - unreachable with current approach.')
    print()
    print('Required CV to hit target: {:.6f} (NEGATIVE - impossible)'.format((target - intercept) / slope))
else:
    print('The intercept is lower than the target.')
    print('Required CV to hit target: {:.6f}'.format((target - intercept) / slope))

KEY INSIGHT: THE INTERCEPT PROBLEM

Intercept: 0.0525
Target: 0.0347

CRITICAL: The intercept is HIGHER than the target!
This means even with CV=0 (perfect training), LB would be ~0.0525
The target is BELOW the intercept - unreachable with current approach.

Required CV to hit target: -0.004130 (NEGATIVE - impossible)
